In [15]:
%matplotlib inline

import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


## Scraping the data from the Top Albums of All Time

### Sputnikmusic
We need to get the content from the top albums of all time list. To do this, we first use Selenium webdriver to select 'all time' from the dropdown list. Then we can extract the content into a BeautifulSoup object.

In [34]:
def get_sput_alltime():
    """
    Returns a Beautiful Soup object containing the contents of the Top Albums of All Time page from sputnikmusic.com.
    Returns None if content fails to load.
    """
    
    current_year = "2020"
    rankings_url = "https://www.sputnikmusic.com/best/albums/" + current_year + "/"

    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.maximize_window()
    driver.get(rankings_url)

    time.sleep(5)
    content = driver.page_source.encode('utf-8').strip()

    if content is not None:
        soup = BeautifulSoup(content, 'html.parser')

        # Find the correct id for the dropdown
        yr_dd = soup.find("span", text=current_year)
        dd_id = yr_dd.parent.parent.get('id')

        # Select 'All Time' from the dropdown
        driver.find_element_by_id(dd_id).click()
        driver.find_element_by_id(dd_id + "_o_2").click()

        # Wait for webpage to load and then create the parse tree from the HTML
        time.sleep(5)
        content_all_time = driver.page_source.encode('utf-8').strip()

        if content_all_time is not None:
            soup_all_time = BeautifulSoup(content_all_time, 'html.parser')
            driver.quit()
            
            return soup_all_time


    driver.quit()
    return None


[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\mattd\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


We create a pandas DataFrame object from the data obtained above and store it in a variable called data.

In [64]:
def create_data_frame(soup):
    """
    Creates the main data frame for the all-time list.
    """
    
    table = soup.find("tr", class_="alt1").parent.parent
    cells = table.find_all("td", class_="blackbox")
    
    common_str = " votes"
    data = []
    for i in range(0, len(cells), 2):
        row = {}
        row['Rank'] = str(int(cells[i].text)) # Remove leading 0's
        
        fonts = cells[i+1].find_all("font")
        row['Artist'] = fonts[0].text 
        row['Album'] = fonts[1].text
        row['Score'] = fonts[3].text
        row['Ratings'] = fonts[4].text[: -len(common_str)]
        
        data.append(row)
    
    return pd.DataFrame(data)
    
data = create_data_frame(soup_all_time)
print(data)
    

    Rank                Artist                                          Album  \
0      1            Pink Floyd                             Wish You Were Here   
1      2  Ludwig van Beethoven             Symphony No. 9 in D minor, Op. 125   
2      3        Charles Mingus            The Black Saint and the Sinner Lady   
3      4                   Nas                                       Illmatic   
4      5           Miles Davis                                   Kind of Blue   
..   ...                   ...                                            ...   
195  196           Suffocation                            Pierced from Within   
196  197         Leonard Cohen                         Songs of Leonard Cohen   
197  198            Koji Kondo  The Legend of Zelda 25th Anniversary Symphony   
198  199             Sigur Ros                                 AgÃ¦tis byrjun   
199  200                Slayer                                 Reign in Blood   

    Score Ratings  
0    4.

Unfortunately, the data on this page does not include the release year or genre(s) of each album, so we need to scrape those from their corresponding album and artist pages respectively. This needs to be done carefully so as to not overload the site with many requests per second.

In [42]:
def get_album_links(soup):
    """
    Returns the album page links for each album in sputnik all time list.
    """
    
    base_url = "https://www.sputnikmusic.com"
    table = soup.find("tr", class_="alt1").parent.parent
    
    links = [urljoin(base_url, a.get('href')) for a in table.find_all("a")]
    return links

In [67]:
links = get_album_links(soup_all_time)
print(len(links))

r = requests.get(links[0])
if r.status_code == 200:
    soup = BeautifulSoup(r.content, "html.parser")
    
    with open("album_page.html", "wb") as file:
        file.write(soup.prettify("utf-8"))


200
